Ej. 4. Evaluación de la técnica de expansión de consultas

Primero instalamos todo el paquete BM25s ejecutando la siguiente orden:

In [ ]:
!pip install bm25s[full]

Ahora importamos las librerías Python necesarias para trabajar inicialmente

In [2]:
import bm25s # To create indexes and search documents using BM25
import Stemmer  # For stemming terms
import json  # To load the JSON-formatted corpus

Descargamos la colección TREC-COVID

In [ ]:
!gdown 1Q1ZnHkstIIQQF8NTPyKQH7sEcpVfThFO
!unzip -o trec-covid-RI.zip

Seguimos con la preparación del corpus

In [4]:
# Parseamos el corpus
with open("corpus.jsonl", "r", encoding="utf-8") as f:
    corpus_lines = f.readlines() # Leemos todas las lineas del JSONL
    corpus_content = [] # Preparamos una lista de objetos JSON
    for linea in corpus_lines: # Cada línea de un JSONL es un objeto JSON...
        corpus_content.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

# Preparamos el corpus en los dos formatos requeridos
corpus_verbatim = list() # Una versión literal que mantenga la estructura original de los "documentos".
corpus_plaintext = list() # Una versión de texto plano para tokenización e indexación.
for entry in corpus_content:
    # BM25S expects 'id', 'title', and 'text'
    document = {"id": entry["_id"], "title": entry["title"].lower(), "text": entry["text"].lower()}
    corpus_verbatim.append(document)
    corpus_plaintext.append(entry["text"].lower())

Creamos un stemmer y tokenizamos el corpus

In [ ]:
# Tokenizamos mediante un stemmer para el idioma inglés (PREPROCESADO Y NORMALIZACIÓN)
stemmer = Stemmer.Stemmer("english")
corpus_tokenized = bm25s.tokenize(corpus_plaintext, stopwords="en", stemmer=stemmer, show_progress=True)

Creamos el retriever con la mejor parametrización obtenida del ejercicio 2 (Eliminación de palabras vacías, aplicando estematización y usando la variante de BM25s e IDF lucene) e indexamos el corpus ya tokenizado

In [ ]:
bm25_flavor = "lucene" # Usamos la implementacion lucene de la funcion del IDF
idf_flavor = "lucene" # Usamos la implementacion lucene de la funcion del IDF

retriever = bm25s.BM25(corpus=corpus_verbatim, method=bm25_flavor, idf_method=idf_flavor)
retriever.index(corpus_tokenized, show_progress=True) # Indexamos el corpus tokenizado

Cargamos las consultas de la colección

In [7]:
queries = []
with open("queries.jsonl", "r", encoding="utf-8") as f:
    queries_lines = f.readlines() # Leemos todas las lineas del JSONL
    for linea in queries_lines: # Cada línea de un JSONL es un objeto JSON...
        queries.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

Ahora implementamos el código para obtener el contenido textual de los n documentos relevantes y sus puntuaciones para cada consulta del conjunto de consultas cargado antes

In [8]:
def submit_queries_and_get_run(queries, stemmer, retriever, stopwordsConfig, max_results=100, retornarIdentificadores=False):
    run = {}

    # Iteramos para enviar cada consulta al retriever para obtener
    # resultados
    for query in queries:
        # Para evaluar después el rendimiento, es útil conservar el id de
        # la consulta
        query_id = query["_id"]

        query_string = query["metadata"]["narrative"].lower()

        # Tokenizamos la consulta. ¡Atención! Debemos tokenizar la consulta
        # con la misma configuración que usamos al indexar el corpus.
        #
        query_tokenized = bm25s.tokenize(query_string, stopwords=stopwordsConfig, stemmer=stemmer, show_progress=False)

        # Retornamos los k resultados más “top” como una tupla con nombre.
        # Por favor, lee la documentación de BM25S para otras alternativas.
        #
        results = retriever.retrieve(query_tokenized, corpus=retriever.corpus, k=max_results, return_as="tuple", show_progress=False)

        # Los documentos y sus puntuaciones se guardan en dos campos
        # diferentes de la tupla.
        #
        returned_documents = results.documents[0]
        relevance_scores = results.scores[0]

        returned_content = []
        for i in range(len(returned_documents)):
##          print(f"\t{i}\t{relevance_scores[i]}\t{returned_documents[i]['id']}\t{returned_documents[i]['title'][0:80]}...")
            if retornarIdentificadores:
              returned_content.append(returned_documents[i]["id"])
            else:
              returned_content.append(returned_documents[i])
        run[query_id] = returned_content

    return run

Implementamos el código para la carga de los juicios de relevancia

In [9]:
def cargarJuiciosDeRelevancia(recursoJuiciosRelevancia):
    # Leer los datos del archivo de texto
    juicios = {}
    saltarLinea = True
    with open(recursoJuiciosRelevancia, 'r') as f:
        for linea in f:
            if saltarLinea:
                saltarLinea = False
                continue
            if linea.strip():  # Saltar líneas vacías
                query_id, corpus_id, score = linea.split()
                if query_id not in juicios:
                    juicios[query_id] = []
                juicios[query_id].append({"corpus_id": corpus_id, "score": int(score)})

    # El formato de los juicios de relevancia necesita ser ligeramente
    # transformado
    relevance_judgements_reformat = dict()
    for query_id in juicios:
        rel_docs = []
        for j in juicios[query_id]:
            rel_docs.append(j["corpus_id"])
        relevance_judgements_reformat[query_id] = rel_docs

    return relevance_judgements_reformat

juiciosRelevancia = cargarJuiciosDeRelevancia("qrels.tsv")

Desarrollamos código auxiliar para la generación de rondas de resultados

In [10]:
import math
from collections import Counter
from heapq import heappush, heappop
from typing import List, Tuple, Any

def obtenerContenidosRelevantes(relevantes):
  contenidoTextual = [] # para cada query guardamos el contenido textual de los documentos resultado

  for i in relevantes: # para cada query tenemos documentos...
    texto = ""
    for j in range(len(relevantes[i])):
      texto += relevantes[i][j]["text"]
    contenidoTextual.append(texto)

  return contenidoTextual

def obtenerFrecuenciasRelevantes(contenidoTextual):
  frecuencias_relevantes = []
  for i in contenidoTextual:
    frecuencias_relevantes.append(calcularTF(i, True))
  return frecuencias_relevantes

def obtenerFrecuenciasRestantes(frecuencias_relevantes):
  frecuencias_relevantes_restantes = []

  for i in range(len(frecuencias_relevantes)):
    frecuencias_relevantes_restantes.append(dict())

  # recorrer los terminos del corpus
  for termino in frecuencias_corpus:
    frecuencia = frecuencias_corpus[termino]
    # para cada termino buscamos si esta en las frecuencias relevantes de cada query
    for i in range(len(frecuencias_relevantes)):
      conjunto_relevantes = frecuencias_relevantes[i]
      # si el termino del corpus no esta en dicho conjunto i-esimo
      if termino not in conjunto_relevantes.keys():
        # añadimos el termino junto con su frecuencia a la lista de frecuencias restantes en la posicion i-esima
        frecuencias_relevantes_restantes[i][termino] = frecuencia
  return frecuencias_relevantes_restantes

def obtenerIdsContenidosRestantes(idsRelevantes):
  idsRestantes = []
  contenidoRestante = ""

  for linea in corpus_verbatim:
    if linea["id"] not in idsRelevantes:
      idsRestantes.append(linea["id"])
      contenidoRestante += linea["text"]

  return idsRestantes, contenidoRestante

def obtenerConjuntoValoresLLRS(frecuencias_relevantes, frecuencias_relevantes_restantes):
  numeroMaximoDeTerminos = float('inf') # para que se incluyan todas las puntuaciones
  limiteDePuntuacion = -float('inf') # para que no haya un valor minimo a la hora de puntuar

  conjuntoValoresLLRSigno = []

  for i in range(len(frecuencias_relevantes)):
    conjuntoValoresLLRSigno.append(LogLikelihood.compare_frequencies(Counter(frecuencias_relevantes[i]), Counter(frecuencias_relevantes_restantes[i]), numeroMaximoDeTerminos, limiteDePuntuacion))

  return conjuntoValoresLLRSigno

def calcularTF(texto, tokenizar):
    if tokenizar:
      tokens = bm25s.tokenize(texto, stopwords="en", stemmer=stemmer, show_progress=False)
    else:
      tokens = texto

    tmp = dict()

    for document in tokens[0]:
        freqs = dict(Counter(document))
        for token, freq in freqs.items():
            try:
                tmp[token] += freq
            except:
                tmp[token] = freq

    inverted_vocab = {tokens[1][key]: key for key in tokens[1].keys()}

    total_freqs = dict()

    for key, freq in dict(tmp).items():
        term = inverted_vocab[key]
        total_freqs[term] = freq

    return total_freqs

class LogLikelihood:
    @staticmethod
    def entropy(*elements: int) -> float:
        """
        Calculates the unnormalized Shannon entropy:
        -sum x_i log x_i / N = -N sum x_i/N log x_i/N
        """
        total = sum(elements)
        result = sum(LogLikelihood._x_log_x(e) for e in elements)
        return LogLikelihood._x_log_x(total) - result

    @staticmethod
    def _x_log_x(x: int) -> float:
        return 0.0 if x == 0 else x * math.log(x)

    @staticmethod
    def log_likelihood_ratio(k11: int, k12: int, k21: int, k22: int) -> float:
        """
        Calculates the raw log-likelihood ratio for two events.
        """
        assert k11 >= 0 and k12 >= 0 and k21 >= 0 and k22 >= 0, "Counts must be non-negative"

        row_entropy = LogLikelihood.entropy(k11 + k12, k21 + k22)
        column_entropy = LogLikelihood.entropy(k11 + k21, k12 + k22)
        matrix_entropy = LogLikelihood.entropy(k11, k12, k21, k22)

        if row_entropy + column_entropy < matrix_entropy:
            return 0.0  # Round-off error

        return 2.0 * (row_entropy + column_entropy - matrix_entropy)

    @staticmethod
    def root_log_likelihood_ratio(k11: int, k12: int, k21: int, k22: int) -> float:
        """
        Calculates the root log-likelihood ratio for two events.
        """
        llr = LogLikelihood.log_likelihood_ratio(k11, k12, k21, k22)
        sqrt = math.sqrt(llr)
        if (k11 / (k11 + k12)) < (k21 / (k21 + k22)):
            sqrt = -sqrt
        return sqrt

    @staticmethod
    def compare_frequencies(a: Counter, b: Counter, max_return: int, threshold: float) -> List[Tuple[Any, float]]:
        """
        Compares two sets of counts to find over-represented items in the first set.
        """
        total_a = sum(a.values())
        total_b = sum(b.values())
        best = []

        def compare_and_add(t):
            k_a = a[t]
            k_b = b[t]
            score = LogLikelihood.root_log_likelihood_ratio(k_a, total_a - k_a, k_b, total_b - k_b)
            if score >= threshold:
                heappush(best, (score, t))
                if len(best) > max_return:
                    heappop(best)

        for t in a.keys():
            compare_and_add(t)

        if threshold < 0:
            for t in b.keys():
                if t not in a:
                    compare_and_add(t)

        # Sort results by score descending
        return sorted(best, key=lambda x: -x[0])

class ScoredItem:
    def __init__(self, item, score):
        self.item = item
        self.score = score

    def __repr__(self):
        return f"ScoredItem(item={self.item}, score={self.score})"

def obtenerLosMTerminosRelevantesConLosValoresMasAltos(conjuntoValoresLLRSigno, m):
  mTerminosRelevantesConLosValoresMasAltos = []

  for i in range(len(queries)):
    mTerminosRelevantesConLosValoresMasAltos.append(dict())

  frecuenciasTerminosQueries = dict()

  index = 0
  for query in queries:
    frecuenciasTerminosQueries[query["_id"]] = calcularTF(queries[index]["metadata"]["narrative"], True)

    numeroDeTerminosRelevantesEncontrados = 0
    indexValoresLLRS = 0
    while indexValoresLLRS < len(conjuntoValoresLLRSigno[index]) and numeroDeTerminosRelevantesEncontrados < m:
      tupla = conjuntoValoresLLRSigno[index][indexValoresLLRS]
      valor = tupla[0]
      termino = tupla[1]

      if termino not in frecuenciasTerminosQueries[query["_id"]].keys():
        mTerminosRelevantesConLosValoresMasAltos[index][termino] = valor
        numeroDeTerminosRelevantesEncontrados += 1

      indexValoresLLRS += 1

    index += 1
  return mTerminosRelevantesConLosValoresMasAltos

def expandirQueries(mTerminosRelevantesConLosValoresMasAltos):
  queriesExpandidas = queries

  index = 0
  for consulta in queriesExpandidas:
    terminosAñadir = ""
    for termino in mTerminosRelevantesConLosValoresMasAltos[index]:
      terminosAñadir += " " + termino
    index += 1
    consulta["metadata"]["narrative"] += terminosAñadir
  return queriesExpandidas

def compute_precision_recall_f1(run, relevance_judgements):
    # Initialize lists to hold precision, recall, and f1 scores for each query
    precision_values = []
    recall_values = []
    f1_values = []

    # Initialize global counts for micro-averaging
    global_retrieved = 0
    global_relevant = 0
    global_retrieved_and_relevant = 0

    # Compute precision, recall, and F1 score for each query
    for query_id in run.keys():
        retrieved_results = run[query_id]
        relevant_results = relevance_judgements[query_id]
        relevant_and_retrieved = set(retrieved_results) & set(relevant_results)

        # Update global counts
        global_retrieved += len(retrieved_results)
        global_relevant += len(relevant_results)
        global_retrieved_and_relevant += len(relevant_and_retrieved)

        # Compute precision and recall
        precision = len(relevant_and_retrieved) / len(retrieved_results) if len(retrieved_results) > 0 else 0
        recall = len(relevant_and_retrieved) / len(relevant_results) if len(relevant_results) > 0 else 0

        # Compute F1 score if both precision and recall are non-zero
        if (precision + recall) > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
            f1_values.append(f1)

        # Append precision and recall for the current query
        precision_values.append(precision)
        recall_values.append(recall)

    # Compute macro-averages
    macro_average_precision = sum(precision_values) / len(precision_values) if precision_values else 0
    macro_average_recall = sum(recall_values) / len(recall_values) if recall_values else 0
    macro_average_f1 = sum(f1_values) / len(f1_values) if f1_values else 0

    macro_average_precision = round(macro_average_precision,3)
    macro_average_recall = round(macro_average_recall,3)
    macro_average_f1 = round(macro_average_f1,3)

    # Compute micro-averages
    micro_average_precision = global_retrieved_and_relevant / global_retrieved if global_retrieved > 0 else 0
    micro_average_recall = global_retrieved_and_relevant / global_relevant if global_relevant > 0 else 0
    micro_average_f1 = (2 * (micro_average_precision * micro_average_recall) /
                        (micro_average_precision + micro_average_recall)) if (micro_average_precision + micro_average_recall) > 0 else 0

    micro_average_precision = round(micro_average_precision,3)
    micro_average_recall = round(micro_average_recall,3)
    micro_average_f1 = round(micro_average_f1,3)

    return macro_average_precision, macro_average_recall, macro_average_f1, micro_average_precision, micro_average_recall, micro_average_f1

def grabarResultados(resultados, nombreFichero, map, mar, maf, mip, mir, mif):
  with open(nombreFichero + ".json", "w") as f:
    #print(resultados)
    resultados["macro_average_precision"] = map
    resultados["macro_average_recall"] = mar
    resultados["macro_average_f1"] = maf
    resultados["micro_average_precision"] = mip
    resultados["micro_average_recall"] = mir
    resultados["micro_average_f1"] = mif
    json.dump(resultados, f)

Y generamos las rondas de resultados evaluando cada configuración con los juicios de relevancia

In [11]:
generacionRondas = False # cambiar este valor para realizar la generación de las rondas

n = list(range(1, 6))
m = list(range(3, 6))

if generacionRondas:
  # Calculamos las frecuencias de los términos en todos los documentos del corpus
  frecuencias_corpus = calcularTF(corpus_tokenized, False)

  for i in n:
    for j in m:
        print("Configuración:\tn =", i, "documentos\tm =", j, "términos")

        relevantes = submit_queries_and_get_run(queries, stemmer, retriever, "en", i)
        contenidoTextual = obtenerContenidosRelevantes(relevantes)

        frecuencias_relevantes = obtenerFrecuenciasRelevantes(contenidoTextual)
        frecuencias_relevantes_restantes = obtenerFrecuenciasRestantes(frecuencias_relevantes)

        conjuntoValoresLLRSigno = obtenerConjuntoValoresLLRS(frecuencias_relevantes, frecuencias_relevantes_restantes)

        mTerminosRelevantes = obtenerLosMTerminosRelevantesConLosValoresMasAltos(conjuntoValoresLLRSigno, j)

        queriesExpandidas = expandirQueries(mTerminosRelevantes)

        resultadosFinales = submit_queries_and_get_run(queriesExpandidas, stemmer, retriever, "en", 100, True)

        map, mar, maf, mip, mir, mif = compute_precision_recall_f1(resultadosFinales, juiciosRelevancia)

        nombreFicheroResultadoAGrabar = "n" + str(i) + "-m" + str(j)
        grabarResultados(resultadosFinales, nombreFicheroResultadoAGrabar, map, mar, maf, mip, mir, mif)

Cargamos los ficheros de resultados generados con cada ejecución del bloque de código anterior

In [ ]:
!gdown 1SJPJTl2NrSURLDNq3BKwLCFzdh9JaOqE
!unzip -o Resultados_obtenidos.zip

Mostramos los resultados obtenidos de cada ejecución procesando los ficheros cargados antes

In [ ]:
for i in n:
    for j in m:
      nombreEjecucion = "n" + str(i) + "-m" + str(j)
      with open(nombreEjecucion + ".json", "r") as f:
        print("------Ejecución: n =", i, "documentos\tm =", j, "términos","------")
        resultados = json.load(f)
        print("----Resultados----")
        print("--Queries--")
        contador = 0
        for query in resultados:
          print("Query:",query, "\tResultados:", resultados[query])
          contador = contador + 1
          if contador == len(queries):
            break
        print("--Rendimiento--")
        print("Macro average precision:", resultados["macro_average_precision"])
        print("Macro average recall:", resultados["macro_average_recall"])
        print("Macro average F1:", resultados["macro_average_f1"])
        print()
        print("Micro average precision:", resultados["micro_average_precision"])
        print("Micro average recall:", resultados["micro_average_recall"])
        print("Micro average F1:", resultados["micro_average_f1"])
        print()
        f.close()